# ASR evaluation
we perform subjective evaluation based on CER/WER by strong ASR

In [1]:
import scipy.io.wavfile as wavfile
import librosa
import os
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt


In [2]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import torch
torch.cuda.set_device(0)
inference_16k_pipline = pipeline(
    task=Tasks.auto_speech_recognition,
    model='damo/speech_paraformer_asr_nat-aishell1-pytorch', device='gpu')

rec_result = inference_16k_pipline(audio_in='https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_audio/asr_example_zh.wav')
print(rec_result)

2023-12-27 16:17:15,587 - modelscope - INFO - PyTorch version 2.1.0+cu118 Found.
2023-12-27 16:17:15,589 - modelscope - INFO - Loading ast index from /home/lixing/.cache/modelscope/ast_indexer
2023-12-27 16:17:15,625 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 631ba47eadab494d26a54974ef83a9e4 and a total number of 946 components indexed
/home/lixing/miniconda3/envs/vibvoice+/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-27 16:17:21,860 - modelscope - WARNING - Model revision not specified, use revision: v1.1.8
2023-12-27 16:17:23,029 - modelscope - INFO - initiate model from /home/lixing/.cache/modelscope/hub/damo/speech_paraformer_asr_nat-aishell1-pytorch
2023-12-27 16:17:23,031 - modelscope - INFO - initiate model from location /home/lixing/.cache/models

Please install rotary_embedding_torch by: 
 pip install -U rotary_embedding_torch
If you want use h5py dataset, please pip install h5py, and try it again


2023-12-27 16:17:26,443 - modelscope - INFO - Decoding with wav files ...
2023-12-27 16:17:26,764 - modelscope - INFO - Computing the result of ASR ...


{'text': '欢迎大家来体验打磨愿推出的语音识别模型'}


In [7]:
'''
prepare processed data into _tmp folder
1. separate to 1-st channel (vibration) or 2-nd channel (microphone)
'''
new_rate = 16000
dataset = '../V2S/'
dataset_target = '../V2S_tmp/'
for speaker in os.listdir(dataset):
    if not os.path.exists(dataset_target + speaker):
        os.mkdir(dataset_target + speaker)
    for date in os.listdir(dataset + speaker):
        # print(speaker, date)
        if not os.path.exists(dataset_target + speaker + "/" + date):
            os.mkdir(dataset_target + speaker + "/" + date)
        for wav in os.listdir(dataset + speaker + "/" + date):
            if wav[-3:] != "wav":
                shutil.copy(dataset + speaker + "/" + date + "/" + wav, dataset_target + speaker + "/" + date + "/" + wav)
                continue
            path = dataset + speaker + "/" + date + "/" + wav
            data, sample_rate = librosa.load(path, mono=False, sr=None)
            samples = round(len(data) * float(new_rate) / sample_rate)
            new_data = librosa.resample(data[1], orig_sr=sample_rate, target_sr=new_rate, scale=True)
            wavfile.write(dataset_target + speaker + "/" + date + "/" + wav, new_rate, new_data.T)

In [10]:
'''
evaluation 
python inference.py
'''
import jiwer
data = '../V2S_tmp/'
output = {}
for speaker in os.listdir(data):
    directory = os.path.join(data, speaker)
    for date in os.listdir(directory):
        hypotheses = []
        references = []
        directory_date = os.path.join(directory, date)
        labels = os.path.join(directory_date, 'labels.txt')
        labels = open(labels, 'r').readlines()
        for label in tqdm(labels):
            l = label.strip().split(' ')
            file = l[0]
            text = ''.join(l[1:])
            file = os.path.join(directory_date, file+'.wav')
            try: 
                rec_result = inference_16k_pipline(audio_in=file,)
                hypotheses.append(rec_result['text'])
                references.append(text)
            except:
                pass
        wer = jiwer.cer(hypotheses, references)
        output[speaker + '_' + date] = round(wer * 100, 2)
import pickle
with open('saved_dict.pkl', 'wb') as f:
    pickle.dump(output, f)

  0%|          | 0/47 [00:00<?, ?it/s]2023-12-27 17:40:39,331 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,409 - modelscope - INFO - Computing the result of ASR ...
2023-12-27 17:40:39,411 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,469 - modelscope - INFO - Computing the result of ASR ...
  6%|▋         | 3/47 [00:00<00:02, 20.63it/s]2023-12-27 17:40:39,471 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,520 - modelscope - INFO - Computing the result of ASR ...
2023-12-27 17:40:39,521 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,569 - modelscope - INFO - Computing the result of ASR ...
 13%|█▎        | 6/47 [00:00<00:01, 25.23it/s]2023-12-27 17:40:39,572 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,620 - modelscope - INFO - Computing the result of ASR ...
2023-12-27 17:40:39,622 - modelscope - INFO - Decoding with wav files ...
2023-12-27 17:40:39,662 - modelscope

In [ ]:

folder = '../ABCS/'
split = 'dev'
data = os.path.join(folder, 'Audio', split)
label_folder = os.path.join(folder, 'script', split)
hypotheses = []
references = []
for labels in os.listdir(label_folder):
    speaker = labels.split('_')[0]
    labels = open(os.path.join(label_folder, labels), 'r').readlines()
    directory = os.path.join(data, speaker)
    for label in tqdm(labels):
        l = label.strip().split(' ')
        file = l[0]
        text = ''.join(l[1:])
        file = os.path.join(directory, file+'.wav')
        rec_result = inference_16k_pipline(audio_in=file,)
        print(rec_result['text'], text)
        hypotheses.append(rec_result['text'])
        references.append(text)
        break

import jiwer
wer = jiwer.cer(hypotheses, references)
print(f"CER: {wer * 100:.2f} %") 